# Dynamic Prompt
<img src="./assets/LC_DynamicPrompts.png">

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OLLAMA_HOST_URL=http://localhost:11434


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass


@dataclass
class RuntimeContext:
    is_employee: bool
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

## Build a Dynamic Prompt
Utilize runtime context and middleware to generate a dynamic prompt.

In [6]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt


@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    if not request.runtime.context.is_employee:
        table_limits = "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        table_limits = ""

    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

Include middleware in `create_agent`.

In [7]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
import os

model = ChatOllama(
    model="granite4:latest",
    temperature=0,
    base_url=os.environ['OLLAMA_HOST_URL']
)

agent = create_agent(
    model=model,
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

In [8]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=False, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (7fcb31f0-331b-49fb-90c0-588e8607384d)
 Call ID: 7fcb31f0-331b-49fb-90c0-588e8607384d
  Args:
    query: SELECT t.title, p.amount_spent FROM Track t JOIN PlaylistTrack pt ON t.id = pt.track_id JOIN Playlist p ON pt.playlist_id = p.id WHERE p.user_name = 'Frank Harris' ORDER BY p.amount_spent DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such column: t.title
[SQL: SELECT t.title, p.amount_spent FROM Track t JOIN PlaylistTrack pt ON t.id = pt.track_id JOIN Playlist p ON pt.playlist_id = p.id WHERE p.user_name = 'Frank Harris' ORDER BY p.amount_spent DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
==========================

In [9]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=True, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (d89a695a-d40d-4d90-bb5a-c12d1018c11c)
 Call ID: d89a695a-d40d-4d90-bb5a-c12d1018c11c
  Args:
    query: SELECT description, amount FROM purchases WHERE buyer_name = 'Frank Harris' ORDER BY amount DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: purchases
[SQL: SELECT description, amount FROM purchases WHERE buyer_name = 'Frank Harris' ORDER BY amount DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (fb2354dc-e070-4fb8-980c-a158d3787afd)
 Call ID: fb2354dc-e070-4fb8-980c-a158d3787afd
  Args:
    query: SELEC

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT